In [ ]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise
import os
from os.path import exists, join, basename, splitext

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

#video_path 
video_path = '/content/drive/MyDrive/Videos/277675141.mp4'

#output directory for json

output_json='/content/drive/MyDrive/posedetection/json_video/'


# #json filepath for similarity check target frame
# json_file='/content/drive/MyDrive/posedetection/keypoints_from_videos_json/squat/squat_000000000170_keypoints.json'

# json_kp_checkpoints='/content/drive/MyDrive/posedetection/selected_frame_5/'
# ''' enter joint index number to measure rep count
#     0='Right elbow'
#     1='Left elbow'
#     2='Right Shoulder'
#     3='Left Shoulder'
#     4='Right Knee'
#     5='Left Knee'
#     6='Right Hip'
#     7='Left Hip'
# '''
joint_index=4

openpose setup (Not to be used during deployment)


In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
# git_repo_url = 'https://github.com/dinatih/openpose.git'
project_name = splitext(basename(git_repo_url))[0]

if 1 or not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949

  print("install new CMake becaue of CUDA10")
  if not exists('cmake-3.13.0-Linux-x86_64.tar.gz'):
    !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("CMakelist.txt's caffe fix")
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
!wget -P /content/openpose/models/pose/coco http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/coco/pose_iter_440000.caffemodel 



!cd openpose && ./build/examples/openpose/openpose.bin --video '{video_path}' --model_pose COCO --write_json '{output_json}' --part_candidates --display 0 --render_pose 0


In [ ]:
frame_no=0
json_processed_output=[]

#variables to initialise rep counter
rep_count = 0
squat_pos = 0
prev_squat_pos = 0
#similarity_check_normalization

def scale_transform_normalize(coords):
    """
    Parameters:
    coords (ndarray): array of (x,y,c) coordinates

    Returns:
    ndarray: coords scaled to 1x1 with center at (0,0)
    ndarray: confidence scores of each joint
    """
    coords, scores = coords[:,:,:-1], coords[:,:,-1]
    diff = coords.max(axis=1) - coords.min(axis=1)
    diff_max = np.max(diff, axis=0)
    mean = coords.mean(axis=1).reshape(coords.shape[0],1,coords.shape[-1])
    out = (coords - mean) / diff_max
    
    return out, scores

# #preprocess similarity check target frame

# final_data=np.array([])
# for filename in sorted(os.listdir(json_kp_checkpoints)):
#     f = open(json_kp_checkpoints+filename,)
#     data = json.load(f)
#     keypoints=list(data.items())[1][1][0]['pose_keypoints_2d']
#     image_keypoints=[]
#     for keys in range(0,len(keypoints),3):
#       image_keypoints.append(np.array([keypoints[keys],keypoints[keys+1],keypoints[keys+2]]))
#     final_data=np.append(final_data,image_keypoints)
#     f.close()
# poses=final_data.reshape(5,18,3)
# poses, scores = scale_transform_normalize(poses)
# poses /= np.linalg.norm(poses, axis=2)[:, :, np.newaxis]
# poses = poses.reshape((poses.shape[0], -1))
# start_pose=poses[0].reshape(1,-1)
# quarter_pose=poses[1].reshape(1,-1)
# mid_pose=poses[2].reshape(1,-1)
# threefourth_pose=poses[3].reshape(1,-1)
# end_pose=poses[4].reshape(1,-1)
# prev_pose=-1
# rep_counter=0
# progress_counter=[]

  


#functions to calcualte max min and mid ROM
def getjointrom_min(json_values,joint_index):
  x=[]
  for i in range(len(json_values)):
    x.append(json_values[i]['angles'][joint_index])
  return np.nanmin(x)

def getjointrom_max(json_values,joint_index):
  x=[]
  for i in range(len(json_values)):
    x.append(json_values[i]['angles'][joint_index])
  return np.nanmax(x)
def getjointrom_mid(json_processed_output,joint_index):
  return np.linspace(json_processed_output[-1]['rom_min_value'][joint_index],json_processed_output[-1]['rom_max_value'][joint_index],3)[1]



for filename in sorted(os.listdir(output_json)):
  
  f = open(output_json+filename,)
  data = json.load(f)
  pose_kp_raw = data['people'][0]['pose_keypoints_2d']
  
  
  # #similarity_check
  # cur_frame=np.array(pose_kp_raw).reshape(1,18,3)
  # cur_frame, scores = scale_transform_normalize(cur_frame)
  # cur_frame /= np.linalg.norm(cur_frame, axis=2)[:, :, np.newaxis]
  # cur_frame = cur_frame.reshape((cur_frame.shape[0], -1))
  
  # #rep counter
  # if pairwise.cosine_similarity(cur_frame,start_pose)[0]>=0.99 and (prev_pose==0 or prev_pose==-1):
  #   progress_counter.append(0)
  #   similarity_frame='start_pose'
  #   similarity_score=pairwise.cosine_similarity(cur_frame,start_pose)[0]
  #   prev_pose=0
  # elif pairwise.cosine_similarity(cur_frame,quarter_pose)[0]>=0.99 and prev_pose==0:
  #   progress_counter.append(0)
  #   similarity_frame='quarter_pose'
  #   similarity_score=pairwise.cosine_similarity(cur_frame,quarter_pose)[0]
  #   prev_pose=1
  #   rep_counter+=1
  # elif pairwise.cosine_similarity(cur_frame,quarter_pose)[0]>=0.99 and prev_pose==1:
  #   progress_counter.append(0.25)
  #   similarity_frame='quarter_pose'
  #   similarity_score=pairwise.cosine_similarity(cur_frame,quarter_pose)[0]
  #   prev_pose=1
  # elif pairwise.cosine_similarity(cur_frame,mid_pose)[0]>=0.99 and prev_pose==1:
  #   progress_counter.append(0.25)
  #   similarity_frame='mid_pose'
  #   similarity_score=pairwise.cosine_similarity(cur_frame,mid_pose)[0]
  #   prev_pose=2
    
  # elif pairwise.cosine_similarity(cur_frame,mid_pose)[0]>=0.99 and prev_pose==2:
  #   progress_counter.append(0.50)
  #   similarity_frame='mid_pose'
  #   similarity_score=pairwise.cosine_similarity(cur_frame,mid_pose)[0]
  #   prev_pose=2
  # elif pairwise.cosine_similarity(cur_frame,threefourth_pose)[0]>=0.99 and prev_pose==2:
  #   progress_counter.append(0.50)
  #   similarity_frame='threefourth_pose'
  #   similarity_score=pairwise.cosine_similarity(cur_frame,threefourth_pose)[0]
  #   prev_pose=3
  # elif pairwise.cosine_similarity(cur_frame,threefourth_pose)[0]>=0.99 and prev_pose==3:
  #   progress_counter.append(0.75)
  #   similarity_frame='threefourth_pose'
  #   similarity_score=pairwise.cosine_similarity(cur_frame,threefourth_pose)[0]
  #   prev_pose=3
  # elif pairwise.cosine_similarity(cur_frame,end_pose)[0]>=0.99 and prev_pose==3:
  #   progress_counter.append(0.75)
  #   similarity_frame='end_pose'
  #   similarity_score=pairwise.cosine_similarity(cur_frame,end_pose)[0]
  #   prev_pose=4
  # elif pairwise.cosine_similarity(cur_frame,end_pose)[0]>=0.99 and prev_pose==4:
  #   progress_counter.append(1)
  #   similarity_frame='end_pose'
  #   similarity_score=pairwise.cosine_similarity(cur_frame,end_pose)[0]
  #   prev_pose=4
  # if progress_counter[-1]==1:
  #   prev_pose=0
  




  # Define Joints
  pose_kp = np.array(pose_kp_raw).reshape([18,3])

  joint_labels=['Right elbow','Left elbow','Right Shoulder','Left Shoulder','Right Knee','Left Knee','Right Hip','Left Hip']
  joints  = [[2,3,4],[5,6,7],[3,2,5],[6,5,2],[8,9,10],[11,12,13],[11,8,9],[8,11,12]]

  # Get array of 3x3 arrays of points from pose_kp corresponding to each joint
  pose_kp_joints = np.array([pose_kp[i,0:2] for i in joints])

  # Define confidence in a joint as quadrature of confidence of each body part
  pose_kp_joints_confidence = np.array([np.sqrt(np.mean(np.square(pose_kp[i,2:3]))) for i in joints])

  # Define vectors with origin at joint
  v1 = pose_kp_joints[:,0,:]-pose_kp_joints[:,1,:]
  v2 = pose_kp_joints[:,2,:]-pose_kp_joints[:,1,:]

  # Compute joint angle
  np.seterr(divide='ignore', invalid='ignore')
  absProb = np.multiply(np.sqrt(np.sum(np.power(v1,2),axis=1)),np.sqrt(np.sum(np.power(v2,2),axis=1)))
  angles  = np.arccos(np.divide(np.sum(np.multiply(v1,v2),axis=1),absProb))*180/np.pi
  
  # Compute cross product sign
  angles_sign = np.sign(np.subtract(np.multiply(v1[:,0],v2[:,1]),np.multiply(v1[:,1],v2[:,0])))

  







  # squat_pos = 1 if int(angles[joint_index]) <= threshold else 0
  # if prev_squat_pos - squat_pos == 1 and similarity_score >= 0.98:
  #   rep_count +=increment_val

  prev_squat_pos = squat_pos
  json_processed_output.append({
      "exercise_id":[filename][0][:-15],
      "frame":frame_no,
      "joint_labels": joint_labels,
      "angles": angles.tolist(),
      "angles_sign": angles_sign.tolist(),
      "confidence": pose_kp_joints_confidence.tolist()})
      #"rep_count":rep_counter,
      # "primary_joint_in_check":joint_labels[joint_index],
      # "similarity_frame":similarity_frame,
      # "similarity_score":similarity_score[0]})
  frame_no+=1
json_processed_output.append({"rom_min_value":[getjointrom_min(json_processed_output,i) for i in range(7)],
                              "rom_max_value":[getjointrom_max(json_processed_output,i) for i in range(7)]})
json_processed_output[-1]['rom_mid_value']=[getjointrom_mid(json_processed_output,i) for i in range(7)]
                              

In [ ]:
json_processed_output

[{'angles': [178.99924671402982,
   154.71002647799764,
   100.39452281666603,
   101.06753836870142,
   179.94669038016477,
   173.23960477426675,
   87.04904176053304,
   99.81567367100413],
  'angles_sign': [1.0, -1.0, -1.0, 1.0, -1.0, -1.0, 1.0, -1.0],
  'confidence': [0.9072211765003432,
   0.8560344041507132,
   0.8945918772214885,
   0.8771386525356182,
   0.8706232379274823,
   0.8521902685320143,
   0.8117498771294435,
   0.8184031749439474],
  'exercise_id': 'squat_000000000000',
  'frame': 0,
  'joint_labels': ['Right elbow',
   'Left elbow',
   'Right Shoulder',
   'Left Shoulder',
   'Right Knee',
   'Left Knee',
   'Right Hip',
   'Left Hip'],
  'primary_joint_in_check': 'Right Knee',
  'rep_count': 0,
  'similarity_frame': 'start_pose',
  'similarity_score': 0.9999089938931733},
 {'angles': [178.61127978816867,
   154.70213223423073,
   100.41319739985116,
   101.04472741959373,
   179.94627000849047,
   173.22616487337845,
   89.90539629380679,
   96.96589281771134],
  

In [ ]:
import json
with open('data.json', 'w') as f:
  json.dump(json_processed_output, f, ensure_ascii=False)